# Creating a vector store

In [1]:
# path to the created database
VECTOR_DB_PATH="/home/vpa/RAGollama3/data/20241006/database"

# The path of the csv file that will be used to create a vectorstore. 
# # The csv will have only one column and each row entry will be treated as a separate document.  
DATA_SOURCE="/home/vpa/RAGollama3/data/20241006/source.csv" 

# The model to use in sentence-transformers for creation embedding
EMBEDDING_MODEL="l3cube-pune/bengali-sentence-similarity-sbert" 

## Vector Store Creation

In [2]:
from sentence_transformers import SentenceTransformer
import torch

# Define the CustomEmbeddings class
class CustomEmbeddings:
    def __init__(self, model_name: str, device: str = None):
        self.device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = SentenceTransformer(model_name)
        self.model.to(self.device)

    def embed_documents(self, texts):
        """Embeds a list of documents (texts) into embeddings."""
        return self.model.encode(texts, convert_to_numpy=True).tolist()

    def embed_query(self, text):
        """Embeds a single query into an embedding."""
        return self.model.encode([text], convert_to_numpy=True).tolist()[0]


/home/vpa/miniconda3/envs/ragollama/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
#---------------------------------
# imports
#---------------------------------
import os
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.schema import Document
from tqdm import tqdm
#--------------------------------------
# global
#--------------------------------------
embedding_model=CustomEmbeddings(EMBEDDING_MODEL)

#--------------------------------------
# helper functions
#--------------------------------------
def create_vector_store(df: pd.DataFrame, 
                        vector_db_path: str) -> None:
    """
    Create a vector store from a dataframe with a single column, where each entry in the dataframe
    is treated as a separate document.

    Args:
        df (pd.DataFrame): DataFrame containing a single column with text data.
        vector_db_path (str): Path to the directory where the vector store will be persisted.
    """
    # Check if GPU is available and set the device
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")

    # Create data directory structure
    os.makedirs(vector_db_path, exist_ok=True)

    # Initialize text splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,
        chunk_overlap=200,
        length_function=len
    )

    
    # Initialize the vector store with GPU-based embeddings
    vectorstore = Chroma(persist_directory=vector_db_path, embedding_function=embedding_model)

    # Process each document in the dataframe
    for data in tqdm(df.iloc[:, 0]):
        documents = [Document(page_content=data)]
        all_splits = text_splitter.split_documents(documents)

        # Add the split documents to the vector store
        vectorstore.add_documents(all_splits)

    # Persist the vector store
    vectorstore.persist()
    print("Vector store created and persisted at:", vector_db_path)


/home/vpa/miniconda3/envs/ragollama/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
df=pd.read_csv(DATA_SOURCE)
df

,text
0,মোটরযানের ফিটনেস নবায়ন প্রক্রিয়া নিবন্ধনের সময়...
1,আয়করের ঠিকানা পরিবর্তন করার নিয়ম : আপনি www.in...
2,সর্বাধিক জিজ্ঞাস্য প্রশ্নাবলি ( FAQ ) ( ক ) প্...
3,"জাতীয় সাইবার নিরাপত্তা এজেন্সি ই - ১৪/এক্স , আ..."
4,""" মৃত ভোটারদের নাম বাদ দেওয়া : মৃত ভোটারের নাম..."
...,...
90,নামজারি সেবা প্রাপ্তি সময় সাধারণ ক্ষেত্রে ২৮ (...
91,নামজারি আবেদন না মঞ্জুর হলে করনীয় বিভিন্ন কারণ...
92,বিডি পুলিশ সাধারণ নির্দেশনা : ০১ . আপনার বাসা ...
93,ফায়ার সার্ভিস ও সিভিল ডিফেন্স স্টেশনসমূহ হতে য...


In [5]:
create_vector_store(df,VECTOR_DB_PATH)

Using device: cuda


/tmp/ipykernel_1024798/2101394220.py:46: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  vectorstore = Chroma(persist_directory=vector_db_path, embedding_function=embedding_model)
100%|██████████| 95/95 [00:01<00:00, 63.83it/s]


Vector store created and persisted at: /home/vpa/RAGollama3/data/20241006/database


/tmp/ipykernel_1024798/2101394220.py:57: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


# Checking Validity of vector store

In [6]:
from langchain.vectorstores import Chroma
from langchain.schema import Document
import torch

def query_vector_store(vector_db_path: str, query_text: str, top_k: int = 5) -> None:
    """
    Query the vector store to find and display the most similar documents to the given query text.

    Args:
        vector_db_path (str): Path to the directory where the vector store is persisted.
        query_text (str): The text query for which to retrieve similar documents.
        top_k (int): The number of top similar documents to retrieve.
    """
    # Load the vector store
    vectorstore = Chroma(persist_directory=vector_db_path, embedding_function=embedding_model)

    # Check if GPU is available and set the device
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")

    # Perform the query
    results = vectorstore.similarity_search(query_text, k=top_k)
    
    # Print the results
    for i, result in enumerate(results):
        print(f"Document {i+1}:")
        print(result.page_content)
        #print("Similarity score:", result.score)
        print()



In [7]:
# Example usage
query_vector_store(vector_db_path=VECTOR_DB_PATH, query_text='মোটরযানের ফিটনেস সনদ কিসের ভিত্তিতে প্রদান করা হয় ?')

Using device: cuda
Document 1:
মোটরযানের ফিটনেস নবায়ন প্রক্রিয়া নিবন্ধনের সময়ই মোটরযানের ধরণ ও তৈরি সনের উপর ভিত্তি করে ৫ বছর , ২ বছর ও ১ বছরের ফিটনেস সনদ প্রদান করা হয় । এরপর মেয়াদ উত্তীর্ণের পূর্বেই মোটরযানের ফিটনেস নবায়নের বাধ্যবাধকতা রয়েছে । বর্তমানে বিআরটিএ'র যে কোনো সার্কেল অফিসে থেকে মোটরযান হাজির করে পরিদর্শক কর্তৃক সন্তোষজনক পরিদর্শন প্রতিবেদন প্রদানপূর্বক নিবন্ধিত মোটরযানের ফিটনেস নবায়ন করা যায় । ঢাকা বিভাগ ও চট্টগ্রাম বিভাগের আওতাধীন সার্কেল অফিসসমূহ থেকে ফিটনেস সনদ নবায়ন করার ক্ষেত্রে অনলাইনে এ্যাপয়েন্টমেন্ট গ্রহণ করতে হয় । এ্যাপয়েন্টমেন্ট গ্রহণের প্রক্রিয়া নিচের লিংকে পাওয়া যাবে । ফিটনেসের জন্য অ্যাপয়েন্টমেন্ট প্রক্রিয়া প্রয়োজনীয় কাগজপত্র : ১ । নির্ধারিত ফরমে পূরণকৃত ও স্বাক্ষরকৃত আবেদনপত্র ( ঢাকা বিভাগ ও চট্টগ্রাম বিভাগের আওতাধীন সার্কেল অফিসসমূহ থেকে ফিটনেস সনদ নবায়ন করার ক্ষেত্রে অনলাইনে এ্যাপয়েন্টমেন্ট গ্রহণ করতে হবে ) ; ২ । প্রয়োজনীয় ফি প্রদানের রশিদ ; ৩ । ফিটনেস সার্টিফিকেটের মূল কপি ; ৪ । হালনাগাদ ট্যাক্স টোকেন এর সত্যায়িত ফটোকপি ; ৫ । TIN সংক্রান্ত কাগজপত্র - এর সত